## Create a generative AI runbook to resolve security findings

## Module 3 - Tool use with human-in-the-loop

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will build on what you learned from Module 1 and Module 2 to complete the following
- Get a finding from Security Hub
- Use Claude Sonnet with **Tool Use** and **Human Approval** to remediate the finding
- Verify the finding is resolved.

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

**In this example, you will get a failed finding from Security Hub for `GuardDuty should be enabled [GuardDuty.1]`**

***

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 1: Get a finding from Security Hub</h3>
    <p>In this step, you will get a failed finding from Security Hub. In this example, we use the AWS SDK for Python (boto3) to access the Security Hub API.</p>
    <p>The Python function will return a JSON representation of the finding. You can also view this finding by visiting Security Hub in the AWS console.</p>
</div>

In [ ]:
# Python code from Module 1. Update to search for
# `GuardDuty should be enabled [GuardDuty.1]`

import boto3
from botocore.exceptions import ClientError
sh = boto3.client('securityhub')


def get_finding(finding_id):

    response = sh.get_findings(
        Filters={
                'ComplianceSecurityControlId': [
                    {
                        'Value': finding_id,
                        'Comparison': 'EQUALS'
                    }
                ],
                'ComplianceStatus':[
                    {
                        'Value': 'FAILED',
                        'Comparison': 'EQUALS'
                    }
                ]
            },
        MaxResults=1

    )
    return response['Findings']

######
# Update get_finding("XXX") with GuardDuty.1
######


guardduty_finding = get_finding("XXX")


######

print(guardduty_finding)

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 2: Create a function to call Claude Sonnet in Amazon Bedrock</h3>
    <p>In this step, you will use the boto3 converse API, to call Claude Sonnet in Amazon Bedrock.</p>
</div>

In [ ]:
# Define get_completion function to call Amazon Bedrock.

modelId = "global.anthropic.claude-sonnet-4-20250514-v1:0"

bedrock_client = boto3.client(service_name='bedrock-runtime')
inference_config = {"temperature": 0}


def get_completion(messages, system_prompt=None, toolConfig=None):

    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config,
    }

    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]

    # Check if tool is provided and add to converse_api_params
    if toolConfig:
        converse_api_params["toolConfig"] = toolConfig

    response = bedrock_client.converse(**converse_api_params)

    return response


print("Get completion function defined.")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 3: Define a tool for Claude Sonnet</h3>
    <p>In this step, you define a function is called `enable_guardduty` that takes a single parameter for Region.</p>
    <p>Then we define the actual Python function that runs when the tool is called (after approval).</p>
</div>

In [ ]:
# Ask for user confirmation before running a tool
def confirm_tool_use(tool_name):
    """Ask for user confirmation before using a tool.
    """

    print("\n⚠️  Claude wants to use the following tool:")
    print(f"    {tool_name}")
    while True:
        response = input("\nDo you want to allow this command? (yes/no): ").strip().lower()
        if response in ['yes', 'y']:
            return True
        elif response in ['no', 'n']:
            return False
        else:
            print("Please answer with 'yes' or 'no'.")


# Define the Tool Specification
guardduty_tool_config = {
    "tools": [
        {
          "toolSpec": {
            "name": "enable_guardduty",
            "description": "Enables Amazon GuardDuty threat detection service in a specified AWS region. Use this if a finding indicates GuardDuty is disabled.",
            "inputSchema": {
              "json": {
                "type": "object",
                "properties": {
                  "region_name": {
                    "type": "string",
                    "description": "The AWS region where GuardDuty should be enabled (e.g., 'us-west-2', 'eu-central-1')."
                  }
                },
                "required": ["region_name"]
              }
            }
          }
        }
     ]
}


def enable_guardduty(region_name):
    """Attempts to enable GuardDuty in the specified region.
    """

    # Before doing anything, get explicit permission from the user
    if not confirm_tool_use("enable_guardduty"):
        result_message = "Command execution denied by user."
        return result_message

    try:
        print(f"Executing: Attempting to enable GuardDuty in region: {region_name}...")
        gd = boto3.client('guardduty', region_name=region_name)
        detectors = gd.list_detectors()

        if not detectors['DetectorIds']:
            print(f"No GuardDuty detector found in {region_name}. Creating a new one...")
            response = gd.create_detector(Enable=True)
            detector_id = response['DetectorId']
            result_message = f"Successfully created and enabled GuardDuty detector {detector_id} in {region_name}."
            print(result_message)
            return result_message
        else:
            detector_id = detectors['DetectorIds'][0]
            detector_status = gd.get_detector(DetectorId=detector_id)
            if detector_status['Status'] == 'ENABLED':
                result_message = f"GuardDuty detector {detector_id} is already enabled in {region_name}."
                print(result_message)
                return result_message
            else:
                print(f"Found existing GuardDuty detector {detector_id}. Enabling it now...")
                gd.update_detector(DetectorId=detector_id, Enable=True)
                result_message = f"Successfully enabled GuardDuty detector {detector_id} in {region_name}."
                print(result_message)
                return result_message

    except boto3.exceptions.Boto3Error as e:
        error_message = f"AWS API Error enabling GuardDuty in {region_name}: {e}"
        print(error_message)
        return error_message  # Return error message for LLM context
    except Exception as e:
        error_message = f"An unexpected error occurred while enabling GuardDuty: {e}"
        print(error_message)
        return error_message  # Return error message for LLM context


# Dictionary mapping tool names to actual functions
available_tools = {
    "enable_guardduty": enable_guardduty
}

print("GuardDuty tool and function defined.")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 4 - Update code to use Claude Sonnet output as tool input</h3>
    <p>In this step, you will call Claude Sonnet and get a response that a tool is requested.</p>
    <p>Claude Sonnet can't use the tools directly, but we can write code that will look for the request, and pass the tool input to our tool function.</p>
</div>


In [ ]:
# For the system variable, enter a prompt to set the context"
# For the prompt variable, update to ask Claude Sonnet to remediate the finding.

######
# Update the system prompt to set the context.
# Update the prompt to ask Claude Sonnet to remediate the finding.
######

system = "You are..."
prompt = f"""

Review the...


<finding>
{guardduty_finding}
</finding>

"""

######

messages = [{"role": "user", "content": [{"text": prompt}]}]
response = get_completion(messages, system, guardduty_tool_config)

# Extract the tool use information
text_output = response['output']['message']['content'][0]['text']
tool_use_id = response['output']['message']['content'][1]['toolUse']['toolUseId']
tool_name = response['output']['message']['content'][1]['toolUse']['name']
tool_input = response['output']['message']['content'][1]['toolUse']['input']

# Print the text and tool use information
print(f"Text output: {text_output}")
print(f"Tool requested: {tool_name}")
print(f"Tool input parameters: {tool_input}")

if tool_name in available_tools:
    function_to_call = available_tools[tool_name]
    tool_result_content = function_to_call(**tool_input)
    print(f"Function execution result: {tool_result_content}")

    # Add the tool call to the prompt
    tool_call = {
        "role": "assistant",
        "content": [
            {"text": text_output},
            {
                "toolUse": {
                    "toolUseId": tool_use_id,
                    "name": tool_name,
                    "input": tool_input
                }
            }
        ]
    }
    messages.append(tool_call)

    # Add the tool results to the prompt
    tool_results_message = {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    "content": [
                        {
                            "text": tool_result_content
                        }
                    ]
                }
            }
        ]
    }
    messages.append(tool_results_message)

    print("Generating final response...\n")
    # Send the complete message after the tool was called back to Claude
    response = get_completion(messages, system, guardduty_tool_config)
    text_output = response['output']['message']['content'][0]['text']
    print(f"Final response: {text_output}")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 5: Verify GuardDuty is enabled.</h3>
    <p>In this step, you will verify GuardDuty is enabled using boto3</p>
</div>

In [ ]:
# Check if GuardDuty is enabled
def is_guardduty_enabled():
    """
    Check if GuardDuty is enabled in the current AWS Region.

    Returns:
        bool: True if GuardDuty is enabled, False otherwise.
    """
    try:
        # Create a GuardDuty client
        guardduty_client = boto3.client('guardduty')

        # List detectors - if GuardDuty is enabled, there will be at least one detector
        response = guardduty_client.list_detectors()

        # Check if any detectors exist
        detector_ids = response.get('DetectorIds', [])

        if detector_ids:
            # Get the status of the first detector
            detector_id = detector_ids[0]
            detector = guardduty_client.get_detector(DetectorId=detector_id)
            return detector.get('Status') == 'ENABLED'
        else:
            # No detectors found, GuardDuty is not enabled
            return False

    except ClientError as e:
        print(f"Error checking GuardDuty status: {e}")
        return False


# Example usage
if __name__ == "__main__":
    if is_guardduty_enabled():
        print("GuardDuty is enabled in the current Region")
    else:
        print("GuardDuty is not enabled in the current Region")